!/usr/bin/env python
coding=utf-8
Copyright 2021 The HuggingFace Inc. team. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and

Pre-Training a 🤗 Wav2Vec2 model on unlabeled audio data

In [ ]:
with open('requirements.txt', 'w', encoding='utf-8') as f:
    f.write('''
accelerate==0.15.0
datasets==2.18.0
huggingface-hub==0.22.2
jiwer==2.5.1
librosa==0.9.2
espnet==202211
espnet_model_zoo==0.1.7
natsort==8.2.0
numpy==1.23.5
omegaconf==2.2.3
pandas==1.5.2
parallel-wavegan==0.5.5
pyctcdecode==0.4.0
soundfile==0.11.0
torch==1.13.0
torchaudio==0.13.0
tqdm==4.64.1
transformers==4.24.0
wandb==0.13.4
''')

In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
import argparse
import math
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Union
import logging

import datasets
import torch
from accelerate import Accelerator
from accelerate.logging import get_logger
from datasets import DatasetDict, concatenate_datasets, load_dataset
import huggingface_hub
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm
from collections import defaultdict

import transformers
from transformers import (
    AdamW,
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    #is_wandb_available,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers.utils import send_example_telemetry

In [ ]:
@dataclass
class DataCollatorForWav2Vec2Pretraining:
    """
    Data collator that will dynamically pad the inputs received and prepare masked indices
    for self-supervised pretraining.

    Args:
        model (:class:`~transformers.Wav2Vec2ForPreTraining`):
            The Wav2Vec2 model used for pretraining. The data collator needs to have access
            to config and ``_get_feat_extract_output_lengths`` function for correct padding.
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`):
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        mask_time_prob (:obj:`float`, `optional`, defaults to :obj:`0.65`):
            Percentage (between 0 and 1) of all feature vectors along the time axis which will be masked for the contrastive task.
            Note that overlap between masked sequences may decrease the actual percentage of masked vectors.
            The default value is taken from the original wav2vec 2.0 article (https://arxiv.org/abs/2006.11477),
            and results in about 49 percent of each sequence being masked on average.
        mask_time_length (:obj:`int`, `optional`, defaults to :obj:`10`):
            Length of each vector mask span to mask along the time axis in the contrastive task. The default value
            originates from the original wav2vec 2.0 article and corresponds to the ``M`` variable mentioned there.
    """

    model: Wav2Vec2ForPreTraining
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"
    pad_to_multiple_of: Optional[int] = None
    mask_time_prob: Optional[float] = 0.65
    mask_time_length: Optional[int] = 10

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # reformat list to dict and set to pytorch format
        batch = self.feature_extractor.pad(
            features,
            padding=self.padding,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        device = batch["input_values"].device
        batch_size = batch["input_values"].shape[0]

        mask_indices_seq_length = self.model._get_feat_extract_output_lengths(batch["input_values"].shape[-1])
        # make sure masked sequence length is a Python scalar
        mask_indices_seq_length = int(mask_indices_seq_length)

        # make sure that no loss is computed on padded inputs
        if batch.get("attention_mask") is not None:
            # compute real output lengths according to convolution formula
            batch["sub_attention_mask"] = self.model._get_feature_vector_attention_mask(
                mask_indices_seq_length, batch["attention_mask"]
            )

        features_shape = (batch_size, mask_indices_seq_length)

        # sample randomly masked indices
        mask_time_indices = _compute_mask_indices(
            features_shape,
            self.mask_time_prob,
            self.mask_time_length,
            attention_mask=batch.get("sub_attention_mask"),
        )

        # sample negative indices
        sampled_negative_indices = _sample_negative_indices(
            features_shape,
            self.model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )
        batch["mask_time_indices"] = torch.tensor(mask_time_indices, dtype=torch.long, device=device)
        batch["sampled_negative_indices"] = torch.tensor(sampled_negative_indices, dtype=torch.long, device=device)

        return batch

In [ ]:
def multiply_grads(params, c):
    """Multiplies grads by a constant *c*."""
    for p in params:
        if p.grad is not None:
            if torch.is_tensor(c):
                c = c.to(p.grad.device)
            p.grad.data.mul_(c)


def get_grad_norm(params, scale=1):
    """Compute grad norm given a gradient scale."""
    total_norm = 0.0
    for p in params:
        if p.grad is not None:
            param_norm = (p.grad.detach().data / scale).norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm**0.5
    return total_norm

In [ ]:
logger = get_logger(__name__)

In [ ]:
def reshape2dict(text):
    parts = text.lstrip('--').split('=')
    return parts[0] + ': ' + parts[-1]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class MagicClass:
    def __init__(self):
        self.args = {
                    'dataset_name': "/content/drive/MyDrive/copt",
                    'dataset_config_names': ['default'],
                    'dataset_split_names': ['train'],
                    'model_name_or_path': 'facebook/wav2vec2-xls-r-300m',
                    'feature_extractor_config': '/content/drive/MyDrive/checkpoints_9/checkpoint-10000/config.json',
                    'output_dir': '/content/drive/MyDrive/checkpoints_10',
                    'max_train_steps': 50000,
                    'num_warmup_steps': 25000,
                    'gradient_accumulation_steps': 1,
                    'learning_rate': 25e-6,
                    'weight_decay': 0.01,
                    'max_duration_in_seconds': 30.0,
                    'min_duration_in_seconds': 1.0,
                    'logging_steps': 100,
                    'saving_steps': 1000,
                    'per_device_train_batch_size': 2,
                    'per_device_eval_batch_size': 2,
                    'adam_beta1': 0.9,
                    'adam_beta2': 0.98,
                    'adam_epsilon': 1e-06,
                    'gradient_checkpointing': True,
                    'wandb_project_name': 'copt-thesis',
                    'wandb_run_name': 'run_10',
                    'validation_split_percentage': 10,
                    'audio_column_name': 'audio',
                    'lr_scheduler_type': 'linear',
                    'max_gumbel_temperature': 2.0,
                    'min_gumbel_temperature': 0.5,
                    'gumbel_temperature_decay': 0.999995,
                    'seed': 0,
                    'mask_time_prob': 0.5,
                    'save_total_limit': 3
                    }

    def __getattr__(self, name):
        return(self.args.get(name))

In [ ]:
args = MagicClass()
is_wandb_available = lambda: True

In [ ]:
# See all possible arguments in src/transformers/args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()
logger.info(accelerator.state)

# Setup logging, we only want one process per machine to log things on the screen.
# accelerator.is_local_main_process is only True for one process per machine.
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()

    # set up weights and biases if available
    if is_wandb_available():
        import wandb

        wandb.init(project=args.wandb_project_name, name=args.wandb_run_name)
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

# Handle the repository creation
if accelerator.is_main_process:
    if args.push_to_hub and not args.preprocessing_only:
        if args.hub_model_id is None:
            repo_name = get_full_repo_name(Path(args.output_dir).name, token=args.hub_token)
        else:
            repo_name = args.hub_model_id
        repo = Repository(args.output_dir, clone_from=repo_name)
    elif args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# 1. Download and create train, validation dataset
# We load all dataset configuration and datset split pairs passed in
# ``args.dataset_config_names`` and ``args.dataset_split_names``
datasets_splits = []
for dataset_config_name, train_split_name in zip(args.dataset_config_names, args.dataset_split_names):
    # load dataset
    dataset_split = load_dataset(
        args.dataset_name, dataset_config_name, split=train_split_name, cache_dir=args.cache_dir
    )
    datasets_splits.append(dataset_split)

# Next, we concatenate all configurations and splits into a single training dataset
raw_datasets = DatasetDict()
if len(datasets_splits) > 1:
    raw_datasets["train"] = concatenate_datasets(datasets_splits).shuffle(seed=args.seed)
else:
    raw_datasets["train"] = datasets_splits[0]

# Take ``args.validation_split_percentage`` from the training dataset for the validation_split_percentage
num_validation_samples = raw_datasets["train"].num_rows * args.validation_split_percentage // 100

if num_validation_samples == 0:
    raise ValueError(
        "`args.validation_split_percentage` is less than a single sample "
        f"for {len(raw_datasets['train'])} training samples. Increase "
        "`args.num_validation_split_percentage`. "
    )

raw_datasets["validation"] = raw_datasets["train"].select(range(num_validation_samples))
raw_datasets["train"] = raw_datasets["train"].select(range(num_validation_samples, raw_datasets["train"].num_rows))


Resolving data files:   0%|          | 0/3099 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# 2. Now we preprocess the datasets including loading the audio, resampling and normalization
# Thankfully, `datasets` takes care of automatically loading and resampling the audio,
# so that we just need to set the correct target sampling rate and normalize the input
# via the `feature_extractor`
if args.feature_extractor_config is not None:
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(args.model_name_or_path, config=args.feature_extractor_config)
else:
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(args.model_name_or_path)

# make sure that dataset decodes audio with correct sampling rate
raw_datasets = raw_datasets.cast_column(
    args.audio_column_name, datasets.features.Audio(sampling_rate=feature_extractor.sampling_rate)
)

# only normalized-inputs-training is supported
if not feature_extractor.do_normalize:
    raise ValueError(
        "Training is only supported for normalized inputs. " "Make sure ``feature_extractor.do_normalize == True``"
    )

# set max & min audio length in number of samples
max_length = int(args.max_duration_in_seconds * feature_extractor.sampling_rate)
min_length = int(args.min_duration_in_seconds * feature_extractor.sampling_rate)

def prepare_dataset(batch):
    sample = batch[args.audio_column_name]

    inputs = feature_extractor(
        sample["array"], sampling_rate=sample["sampling_rate"], max_length=max_length, truncation=True
    )
    batch["input_values"] = inputs.input_values[0]
    batch["input_length"] = len(inputs.input_values[0])

    return batch

# load via mapped files via path
cache_file_names = None
if args.train_cache_file_name is not None:
    cache_file_names = {"train": args.train_cache_file_name, "validation": args.validation_cache_file_name}

# load audio files into numpy arrays
with accelerator.main_process_first():
    vectorized_datasets = raw_datasets.map(
        prepare_dataset,
        num_proc=args.preprocessing_num_workers,
        remove_columns=raw_datasets["train"].column_names,
        cache_file_names=cache_file_names,
    )

    if min_length > 0.0:
        vectorized_datasets = vectorized_datasets.filter(
            lambda x: x > min_length,
            num_proc=args.preprocessing_num_workers,
            input_columns=["input_length"],
        )

    vectorized_datasets = vectorized_datasets.remove_columns("input_length")

# for large datasets it is advised to run the preprocessing on a
# single machine first with ``args.preprocessing_only`` since there will mostly likely
# be a timeout when running the script in distributed mode.
# In a second step ``args.preprocessing_only`` can then be set to `False` to load the
# cached dataset
if args.preprocessing_only:
    raise AssertionError()

In [ ]:
# 3. Load model
if args.feature_extractor_config is not None:
    config = Wav2Vec2Config.from_pretrained(args.feature_extractor_config)
else:
    config = Wav2Vec2Config.from_pretrained(args.model_name_or_path)
# pretraining is only supported for "newer" stable layer norm architecture
# apply_spec_augment has to be True, mask_feature_prob has to be 0.0

if not config.do_stable_layer_norm or config.feat_extract_norm != "layer":
    raise ValueError(
        "PreTraining is only supported for ``config.do_stable_layer_norm=True`` and ``config.feat_extract_norm='layer'"
    )

# initialize model
if args.feature_extractor_config is not None:
    model = Wav2Vec2ForPreTraining.from_pretrained(args.model_name_or_path, config=args.feature_extractor_config)
else:
    model = Wav2Vec2ForPreTraining.from_pretrained(args.model_name_or_path)

# Number of trainable parameters
print(f'Total model parameters: {sum(p.numel() for p in model.parameters())}')
print(f'Trainable model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

# Activate gradient checkpointing if needed
if args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

In [ ]:
# 4. Define data collator, optimizer and scheduler
data_collator = DataCollatorForWav2Vec2Pretraining(
    model=model, feature_extractor=feature_extractor, pad_to_multiple_of=args.pad_to_multiple_of
)
train_dataloader = DataLoader(
    vectorized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=args.per_device_train_batch_size,
)
eval_dataloader = DataLoader(
    vectorized_datasets["validation"], collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
)

# Optimizer
optimizer = torch.optim.AdamW(
    list(model.parameters()),
    lr=args.learning_rate,
    betas=[args.adam_beta1, args.adam_beta2],
    eps=args.adam_epsilon,
)

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)

if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

In [ ]:
# 5. Train
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(vectorized_datasets['train'])}")
logger.info(f"  Num Epochs = {args.num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")
completed_steps = 0

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
for epoch in range(args.num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        # compute num of losses
        num_losses = batch["mask_time_indices"].sum()
        sub_attention_mask = batch.pop("sub_attention_mask", None)
        sub_attention_mask = (
            sub_attention_mask if sub_attention_mask is not None else torch.ones_like(batch["mask_time_indices"])
        )
        percent_masked = num_losses / sub_attention_mask.sum()

        # forward
        outputs = model(**batch)

        # divide loss by gradient accumulation steps since gradients
        # are accumulated for multiple backward passes in PyTorch
        loss = outputs.loss / args.gradient_accumulation_steps
        accelerator.backward(loss)

        # make sure that `num_losses` is summed for distributed training
        # and average gradients over losses of all devices
        if accelerator.state.num_processes > 1:
            num_losses = accelerator.gather(num_losses).sum()
            gradient_multiplier = accelerator.state.num_processes / num_losses
            multiply_grads(model.module.parameters(), gradient_multiplier)
        else:
            multiply_grads(model.parameters(), 1 / num_losses)

        # update step
        if (step + 1) % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:

            # compute grad norm for monitoring
            scale = (
                accelerator.scaler._scale.item()
                if hasattr(accelerator, "scaler") and accelerator.scaler is not None
                else 1
            )
            if accelerator.state.num_processes > 1:
                grad_norm = get_grad_norm(model.module.parameters(), scale)
            else:
                grad_norm = get_grad_norm(model.parameters(), scale)

            # update parameters
            optimizer.step()
            optimizer.zero_grad()

            if not accelerator.optimizer_step_was_skipped:
                lr_scheduler.step()
            elif accelerator.is_local_main_process:
                progress_bar.write(
                    "Gradients have overflown - skipping update step... " f"Updating gradient scale to {scale}..."
                )

            # update gumbel temperature
            gumbel_temperature = max(
                args.max_gumbel_temperature * args.gumbel_temperature_decay**completed_steps,
                args.min_gumbel_temperature,
            )
            if hasattr(model, "module"):
                model.module.set_gumbel_temperature(gumbel_temperature)
            else:
                model.set_gumbel_temperature(gumbel_temperature)

            progress_bar.update(1)
            completed_steps += 1

        # 6. Log all results
        if completed_steps % args.logging_steps == 0 \
                and (step + 1) % args.gradient_accumulation_steps == 0:
            loss.detach()
            outputs.contrastive_loss.detach()
            outputs.diversity_loss.detach()

            if accelerator.state.num_processes > 1:
                loss = accelerator.gather(loss).sum()
                outputs.contrastive_loss = accelerator.gather(outputs.contrastive_loss).sum()
                outputs.diversity_loss = accelerator.gather(outputs.diversity_loss).sum()
                percent_masked = accelerator.gather(percent_masked).sum()

            train_logs = {
                "completed_steps": torch.tensor(completed_steps),
                "loss": (loss * args.gradient_accumulation_steps) / num_losses,
                "constrast_loss": outputs.contrastive_loss / num_losses,
                "div_loss": outputs.diversity_loss / num_losses,
                "%_mask_idx": percent_masked / accelerator.num_processes,
                "ppl": outputs.codevector_perplexity,
                "lr": torch.tensor(optimizer.param_groups[0]["lr"]),
                "temp": torch.tensor(gumbel_temperature),
                "grad_norm": torch.tensor(grad_norm),
            }
            log_str = ""
            for k, v in train_logs.items():
                log_str += "| {}: {:.3e}".format(k, v.item())

            if accelerator.is_local_main_process:
                progress_bar.write(log_str)
                if is_wandb_available():
                  wandb.log(train_logs)

        # save model every `args.saving_steps` steps
        ckpt_prefix = 'checkpoint-'
        if completed_steps % args.saving_steps == 0 \
                and (step + 1) % args.gradient_accumulation_steps == 0:
            if (args.push_to_hub and epoch < args.num_train_epochs - 1) or args.output_dir is not None:
                accelerator.wait_for_everyone()
                unwrapped_model = accelerator.unwrap_model(model)
                #os.makedirs(os.path.join(args.output_dir, ckpt_prefix+str(step + 1)), exist_ok=True)
                unwrapped_model.save_pretrained(os.path.join(args.output_dir, ckpt_prefix+str(completed_steps)),
                                                save_function=accelerator.save)

            if (args.push_to_hub and epoch < args.num_train_epochs - 1) and accelerator.is_main_process:
                repo.push_to_hub(
                    commit_message=f"Training in progress step {completed_steps}",
                    blocking=False,
                    auto_lfs_prune=True,
                )

        # if completed steps > `args.max_train_steps` stop
        if completed_steps >= args.max_train_steps:
            break

    # 7. Validate!
    model.eval()

    # init logs
    val_logs = {
        "val_loss": 0,
        "val_contrastive_loss": 0,
        "val_diversity_loss": 0,
        "val_num_losses": 0,
    }
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch.pop("sub_attention_mask", None)
            outputs = model(**batch)

        val_logs["val_loss"] += outputs.loss
        val_logs["val_contrastive_loss"] += outputs.contrastive_loss
        val_logs["val_diversity_loss"] += outputs.diversity_loss
        val_logs["val_num_losses"] += batch["mask_time_indices"].sum()

    # sum over devices in multi-processing
    if accelerator.num_processes > 1:
        val_logs = {k: accelerator.gather(v).sum() for k, v in val_logs.items()}

    val_logs = {k: v / val_logs["val_num_losses"] for k, v in val_logs.items()}

    log_str = ""
    for k, v in val_logs.items():
        log_str += "| {}: {:.3e}".format(k, v.item())

    if accelerator.is_local_main_process:
        progress_bar.write(log_str)
        if is_wandb_available():
            wandb.log(val_logs)

    '''
    if args.output_dir is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)
        if accelerator.is_main_process:
            if args.push_to_hub:
                repo.push_to_hub(commit_message="End of training", auto_lfs_prune=True)
    '''

INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 2790
INFO:__main__:  Num Epochs = 36
INFO:__main__:  Instantaneous batch size per device = 2
INFO:__main__:  Total train batch size (w. parallel, distributed & accumulation) = 2
INFO:__main__:  Gradient Accumulation steps = 1
INFO:__main__:  Total optimization steps = 50000


  0%|          | 0/50000 [00:00<?, ?it/s]

| completed_steps: 1.000e+02| loss: 3.634e+00| constrast_loss: 3.546e+00| div_loss: 8.857e-01| %_mask_idx: 4.517e-01| ppl: 7.313e+01| lr: 1.000e-07| temp: 1.999e+00| grad_norm: 8.119e+00
| completed_steps: 2.000e+02| loss: 2.929e+00| constrast_loss: 2.849e+00| div_loss: 7.986e-01| %_mask_idx: 5.044e-01| ppl: 1.289e+02| lr: 2.000e-07| temp: 1.998e+00| grad_norm: 5.096e+00
| completed_steps: 3.000e+02| loss: 3.895e+00| constrast_loss: 3.815e+00| div_loss: 8.053e-01| %_mask_idx: 4.675e-01| ppl: 1.246e+02| lr: 3.000e-07| temp: 1.997e+00| grad_norm: 1.290e+01
| completed_steps: 4.000e+02| loss: 3.200e+00| constrast_loss: 3.122e+00| div_loss: 7.795e-01| %_mask_idx: 4.760e-01| ppl: 1.411e+02| lr: 4.000e-07| temp: 1.996e+00| grad_norm: 4.434e+00
| completed_steps: 5.000e+02| loss: 4.077e+00| constrast_loss: 3.995e+00| div_loss: 8.208e-01| %_mask_idx: 4.854e-01| ppl: 1.147e+02| lr: 5.000e-07| temp: 1.995e+00| grad_norm: 9.984e+00
| completed_steps: 6.000e+02| loss: 3.284e+00| constrast_loss: 3.

Configuration saved in /content/drive/MyDrive/checkpoints_10/checkpoint-1000/config.json


| completed_steps: 1.000e+03| loss: 3.624e+00| constrast_loss: 3.539e+00| div_loss: 8.511e-01| %_mask_idx: 5.043e-01| ppl: 9.527e+01| lr: 1.000e-06| temp: 1.990e+00| grad_norm: 5.037e+00


...

Configuration saved in /content/drive/MyDrive/checkpoints_10/checkpoint-4000/config.json


| completed_steps: 4.000e+03| loss: 2.584e+00| constrast_loss: 2.503e+00| div_loss: 8.038e-01| %_mask_idx: 5.071e-01| ppl: 1.256e+02| lr: 4.000e-06| temp: 1.960e+00| grad_norm: 4.441e+00


Model weights saved in /content/drive/MyDrive/checkpoints_10/checkpoint-4000/pytorch_model.bin


| completed_steps: 4.100e+03| loss: 2.779e+00| constrast_loss: 2.695e+00| div_loss: 8.365e-01| %_mask_idx: 4.545e-01| ppl: 1.046e+02| lr: 4.100e-06| temp: 1.959e+00| grad_norm: 6.922e+00
| val_loss: 2.372e+00| val_contrastive_loss: 2.288e+00| val_diversity_loss: 8.308e-01| val_num_losses: 1.000e+00
| completed_steps: 4.200e+03| loss: 3.227e+00| constrast_loss: 3.140e+00| div_loss: 8.737e-01| %_mask_idx: 5.122e-01| ppl: 8.080e+01| lr: 4.200e-06| temp: 1.958e+00| grad_norm: 6.237e+00


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 16>:33                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1316 in backward               │
│                                                                                                  │
│   1313 │   │   elif self.scaler is not None:                                                     │
│   1314 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1315 │   │   else:                                                                             │
│ ❱ 1316 │   │   │   loss.backward(**kwargs)                                                       │
│   1317 │                                                                                         │
│   1318 │   def unscale_gradients(self, optimizer=None):                                          │
│   1319 │   │   """                                                                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:197 in backward               │
│                                                                                                  │
│   194 │   # The reason we repeat same the comment below is that                                  │
│   195 │   # some Python versions print out the first line of a multi-line function               │
│   196 │   # calls in the traceback and some print out the last line                              │
│ ❱ 197 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   198 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   199 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   200                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt